In [22]:
import os
import getpass
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain import PromptTemplate
from langchain import hub
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough


def load_env():
    loaded_env = load_dotenv(dotenv_path=".env", override=True)
    print("Loaded env variables from current directory -> ", loaded_env)
    if loaded_env == False:
        loaded_env = load_dotenv(dotenv_path="../.env", override=True)
        print("Loaded env variables from parent directory -> ", loaded_env)
load_env()

os.environ['GOOGLE_API_KEY']

# If there is no environment variable set for the API key, you can pass the API
    # key to the parameter `google_api_key` of the `GoogleGenerativeAIEmbeddings`
    # function: `google_api_key = "key"`.

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

Loaded env variables from current directory ->  False
Loaded env variables from parent directory ->  True


In [23]:
PERSIST_DIR = "./chroma_vectordb"
import nest_asyncio
from langchain.document_loaders import WebBaseLoader
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma


def retrieve_docs(weburl: str, is_sitemap_url=False): 
    

    nest_asyncio.apply()
    
    loader = None 
    if is_sitemap_url: 
        loader = SitemapLoader(web_path=weburl)
    else:
        loader = WebBaseLoader(web_path=weburl)
    docs = loader.load()
    
    langchain_docs = []
    for doc in docs:
        text_content = doc.page_content
        #print( type(text_content) )
        # The text content between the substrings "code, audio, image and video." to
        # "Cloud TPU v5p" is relevant for this tutorial. You can use Python's `split()`
        # to select the required content.
        #text_content_1 = text_content.split("code, audio, image and video.",1)[1]
        #final_text = text_content_1.split("Cloud TPU v5p",1)[0]
        final_text = text_content
        # Convert the text to LangChain's `Document` format
        langchain_docs.append(Document(page_content=final_text, metadata={"source": "local"}))
    print("No of documents retrieved : ", len(langchain_docs) )
    # Save to disk
    vectorstore = Chroma.from_documents(
                     documents=langchain_docs,                 # Data
                     embedding=gemini_embeddings,    # Embedding model
                     persist_directory=PERSIST_DIR # Directory to save data
                     )
    vectorstore.persist()
    print("No of Documents stored  : " , len(langchain_docs))
    return langchain_docs

In [24]:
web_url = "https://blog.google/technology/ai/google-gemini-ai/"
ret_docs = retrieve_docs(web_url)


No of documents retrieved :  1
No of Documents stored  :  1


In [25]:
web_url = "https://docs.smith.langchain.com/sitemap.xml"
ret_docs = retrieve_docs(web_url, is_sitemap_url=True)

Fetching pages: 100%|#####################################################################################################################################################################################| 219/219 [00:08<00:00, 25.42it/s]


No of documents retrieved :  219


GoogleGenerativeAIError: Error embedding content: 504 Deadline Exceeded

In [ ]:
def get_retriever(): 
    # Load from disk
    vectorstore_disk = Chroma(
                            persist_directory=PERSIST_DIR,       # Directory of db
                            embedding_function=gemini_embeddings   # Embedding model
                       )
    # Get the Retriever interface for the store to use later.
    # When an unstructured query is given to a retriever it will return documents.
    # Read more about retrievers in the following link.
    # https://python.langchain.com/docs/modules/data_connection/retrievers/
    #
    # Since only 1 document is stored in the Chroma vector store, search_kwargs `k`
    # is set to 1 to decrease the `k` value of chroma's similarity search from 4 to
    # 1. If you don't pass this value, you will get a warning.
    retriever = vectorstore_disk.as_retriever(search_kwargs={"k": 3})
    
    # Check if the retriever is working by trying to fetch the relevant docs related
    # to the word 'MMLU' (Massive Multitask Language Understanding). If the length is greater than zero, 
    # it means that the retriever is functioning well.
    print(len(retriever.get_relevant_documents("MMLU")))
    return retriever

In [ ]:
def get_llm(): 
    from langchain_google_genai import ChatGoogleGenerativeAI
    # If there is no environment variable set for the API key, you can pass the API
    # key to the parameter `google_api_key` of the `ChatGoogleGenerativeAI` function:
    # `google_api_key="key"`.
    llm = ChatGoogleGenerativeAI(model="gemini-pro",
                     temperature=0.7, top_p=0.85)
    return llm

In [ ]:
def get_prompt():
    # Prompt template to query Gemini
    llm_prompt_template = """You are an assistant for question-answering tasks.
    Use the following context to answer the question.
    If you don't know the answer, just say that you don't know.
    Use five sentences maximum and keep the answer concise.\n
    Question: {question} \nContext: {context} \nAnswer:"""
    
    llm_prompt = PromptTemplate.from_template(llm_prompt_template)
    
    print(llm_prompt)
    return llm_prompt

In [ ]:
retriever = get_retriever()
llm = get_llm()
llm_prompt = get_prompt()

In [ ]:


# Combine data from documents to readable string format.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create stuff documents chain using LCEL.
# This is called a chain because you are chaining together different elements
# with the LLM. In the following example, to create the stuff chain, you will
# combine the relevant context from the website data matching the question, the
# LLM model, and the output parser together like a chain using LCEL.
#
# The chain implements the following pipeline:
# 1. Extract the website data relevant to the question from the Chroma
#    vector store and save it to the variable `context`.
# 2. `RunnablePassthrough` option to provide `question` when invoking
#    the chain.
# 3. The `context` and `question` are then passed to the prompt where they
#    are populated in the respective variables.
# 4. This prompt is then passed to the LLM (`gemini-pro`).
# 5. Output from the LLM is passed through an output parser
#    to structure the model's response.
def get_dynamic_chain(retriever, format_docs, llm_prompt, llm): 
    return (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | llm_prompt
        | llm
        | StrOutputParser()
    )

chain = get_dynamic_chain(retriever, format_docs, llm_prompt, llm)
    

In [26]:
chain.invoke("What is Gemini?")

"I'm sorry, but this article does not contain the answer to your question about what Gemini is."

In [27]:
chain.invoke("What is langchain?")

'I cannot find the answer to your question in the provided context.'

In [ ]:
chain.invoke("What is langsmith?")

In [ ]:
chain.invoke("What is llama?")

In [ ]:
chain.invoke("which env variables needed for langsmith?")

In [ ]:
chain.invoke("how to add evaluaters for LLM?")

In [ ]:
chain.invoke("what is chain in langchain?")

In [ ]:
chain.invoke("is langsmith free to use for enterprise?")